# Day2

In [80]:
import spacy
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS
import pandas as pd
from sklearn.cluster import KMeans
import time

voc = set()

In [81]:
%run src/file_utils.py
%run src/configuration.py

In [82]:
TYPE = 'type'
PARAGRAPH = 'paragraph'
CONTENT = 'content'
#FILE_PATH = 'data/jsons/'

In [83]:
def readContentOfFile(file_name):
    content = ''
    try:
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    content += item[CONTENT]
    except:
        FileUtils.fix_json(file_name)
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    content += item[CONTENT]
    
    return content

# Clustering without lemmatization

In [97]:
nlp = spacy.load("de")
def perform_lemmatization(document):
    content_of_document = readContentOfFile(document)
    
    #remove minus sign and next-line sign
    content_of_document = content_of_document.replace('-\n','')
    content_of_document = content_of_document.replace('\n',' ')
    
    #replace all gco2 with co2
    content_of_document = content_of_document.replace('gCO2','CO2')
    
    #remove the character we don't need
    remove_char = content_of_document.maketrans('-',' ','+*<>%/&$')
    content_of_document = content_of_document.translate(remove_char)
    
    #adjust the length before tokenization
    content_of_document = content_of_document[0:999999]
    
    sentence = nlp(content_of_document)
    filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
    filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
    filtered_words_withoutpunc = [word for word in filtered_words_withoutdigits if word.pos_ != 'PUNCT']
    filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]
    
    final = []  
    for item in filtered_lemmas:
        #remove the words contain digit except of co2
        if(any(c.isdigit() for c in item)):
            if 'CO2' in item:
                final.append(item)
        else:
            #remove the words contain dot
            if '.' not in item:
                final.append(item)
    
    
    for item in final:
        if item not in voc:
            voc.add(item.lower())
    
    
    lemmatized_content = " ".join(item for item in final)
    
    
    #output the result into file 
    if document.startswith(FILE_PATH):
        filename = "filtered_" + document[len(FILE_PATH):]
    
    with open(filename, 'w') as outfile:
        json.dump(lemmatized_content.lower(), outfile)
    
    
    return lemmatized_content.lower()

# BMW 2010-2017

In [86]:
voc=set()

In [87]:
bmw_lemm_docs_prep = [
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2010.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2011.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2012.json'),
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2013.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2014.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2015.json'),
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2016.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2017.json')]

In [88]:
vectorizer_bmw = TfidfVectorizer(vocabulary=voc)
start_time = time.time()
tfidf_matrix_bmw = vectorizer_bmw.fit_transform(bmw_lemm_docs_prep)
print (time.time() - start_time)

0.20790743827819824


In [89]:
#bmw_lemm_docs_prep

In [90]:
bmw_feature_names = vectorizer_bmw.get_feature_names()
bmw_corpus_index = [n for n in [
    'BMW-2010', 'BMW-2011', 'BMW-2012', 
    'BMW-2013', 'BMW-2014', 'BMW-2015',
    'BMW-2016', 'BMW-2017']]
idf = vectorizer_bmw.idf_
df = pd.DataFrame(tfidf_matrix_bmw.T.todense(), index=bmw_feature_names, columns=bmw_corpus_index)
df['idf'] = idf

In [91]:
df = df.sort_values(by=['BMW-2016'], ascending=False)
df[(df['idf'] != 1)].head(10)
#df.head(10)

,BMW-2010,BMW-2011,BMW-2012,BMW-2013,BMW-2014,BMW-2015,BMW-2016,BMW-2017,idf
next,0.000000,0.000000,0.000000,0.000000,0.000000,0.007599,0.032596,0.032596,1.810930
co2,0.000000,0.000000,0.000000,0.001186,0.001169,0.000000,0.027337,0.027337,1.587787
ergebnisauswirkungen,0.000000,0.000000,0.000000,0.002100,0.016557,0.018676,0.021998,0.021998,1.405465
zweijährig,0.000000,0.000000,0.000000,0.000000,0.019874,0.018878,0.021124,0.021124,1.587787
betrachtungszeitraum,0.000996,0.000000,0.000896,0.002506,0.009876,0.017198,0.020119,0.020119,1.117783
there,0.000000,0.000000,0.000000,0.000000,0.000000,0.024064,0.019841,0.019841,1.810930
brexit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018066,0.018066,2.098612
here,0.000000,0.000000,0.000000,0.000000,0.000000,0.018998,0.017006,0.017006,1.810930
plug,0.000000,0.003687,0.000896,0.001671,0.009876,0.007036,0.016620,0.016620,1.117783
aktienbasierte,0.000000,0.013826,0.017026,0.016705,0.014814,0.016417,0.016620,0.016620,1.117783


# Deutsche Bank 2010-2016

In [107]:
voc=set()

In [108]:
db_lemm_docs_prep = [
     perform_lemmatization(FILE_PATH + 'DeutscheBank-AnnualReport-2010.json'), 
     perform_lemmatization(FILE_PATH + 'DeutscheBank-AnnualReport-2011.json'), 
     perform_lemmatization(FILE_PATH + 'DeutscheBank-AnnualReport-2012.json'),
     perform_lemmatization(FILE_PATH + 'DeutscheBank-AnnualReport-2013.json'), 
     perform_lemmatization(FILE_PATH + 'DeutscheBank-AnnualReport-2014.json'), 
     perform_lemmatization(FILE_PATH + 'DeutscheBank-AnnualReport-2015.json'),
     perform_lemmatization(FILE_PATH + 'DeutscheBank-AnnualReport-2016.json')]

In [109]:
vectorizer_db = TfidfVectorizer(vocabulary=voc)
start_time = time.time()
tfidf_matrix_db = vectorizer_db.fit_transform(db_lemm_docs_prep)
print (time.time() - start_time)

0.4131004810333252


In [113]:
#db_lemm_docs_prep
#voc

In [114]:
db_feature_names = vectorizer_db.get_feature_names()
db_corpus_index = [n for n in [
    'DB-2010', 'DB-2011', 'DB-2012', 
    'DB-2013', 'DB-2014', 'DB-2015',
    'DB-2016']]
idf = vectorizer_db.idf_
df_db = pd.DataFrame(tfidf_matrix_db.T.todense(), index=db_feature_names, columns=db_corpus_index)
df_db['idf'] = idf

In [115]:
df_db = df_db.sort_values(by=['DB-2016'], ascending=False)
df_db[(df_db['idf'] != 1)].head(10)

,DB-2010,DB-2011,DB-2012,DB-2013,DB-2014,DB-2015,DB-2016,idf
beizulegenden,0.000000,0.209947,0.161107,0.117547,0.042820,0.137138,0.149401,1.133531
zeitwerts,0.000000,0.069982,0.043938,0.033334,0.002745,0.045897,0.050533,1.133531
pwcc,0.000000,0.000000,0.000000,0.000000,0.000000,0.006764,0.046072,1.980829
crrcrd,0.000000,0.000000,0.000000,0.047021,0.103942,0.038724,0.044163,1.470004
ncou,0.000000,0.000000,0.039265,0.063776,0.047396,0.034550,0.042430,1.287682
gm,0.000000,0.000000,0.000000,0.000000,0.000000,0.004832,0.037434,1.980829
cib,0.022867,0.022116,0.001331,0.000000,0.000000,0.004397,0.030574,1.287682
kernkapital,0.000516,0.000000,0.005858,0.007603,0.026900,0.025990,0.030210,1.133531
vollumsetzung,0.000000,0.000000,0.000000,0.004550,0.025630,0.018645,0.028492,1.470004
harte,0.000000,0.000000,0.000000,0.000000,0.000000,0.026091,0.025916,1.980829


# Visualization

In [73]:
import plotly as py
import plotly.graph_objs as go
import numpy as np

py.offline.init_notebook_mode(connected=True)

## BMW

In [116]:
key = ['husqvarna', 'aktienbasierte', 'citroën', 'electrification', 'amsterdam', 'drivenow', 'co2', 'brexit', 'here', 'there']

In [122]:
# get the value of certain row
y1 = df.loc['husqvarna'].tolist()
#y2 = df.loc['aktienbasierte'].tolist()
y3 = df.loc['citroën'].tolist()
#y4 = df.loc['electrification'].tolist()
y5 = df.loc['amsterdam'].tolist()
y6 = df.loc['drivenow'].tolist()
y7 = df.loc['co2'].tolist()
y8 = df.loc['brexit'].tolist()
y9 = df.loc['here'].tolist()
y10= df.loc['there'].tolist()

In [123]:
years = np.linspace(2010, 2017, 8)

line1 = go.Scatter(x=years, y=y1, mode='lines+markers', name=key[0])
#line2 = go.Scatter(x=years , y=y2, mode='lines+markers', name=key[1])
line3 = go.Scatter(x=years, y=y3, mode='lines+markers', name=key[2])
#line4 = go.Scatter(x=years, y=y4, mode='lines+markers', name=key[3])

line5 = go.Scatter(x=years, y=y5, mode='lines+markers', name=key[4])
line6 = go.Scatter(x=years, y=y6, mode='lines+markers', name=key[5])
line7 = go.Scatter(x=years, y=y7, mode='lines+markers', name=key[6])
line8 = go.Scatter(x=years, y=y8, mode='lines+markers', name=key[7])
line9 = go.Scatter(x=years, y=y9, mode='lines+markers', name=key[8])
line10= go.Scatter(x=years, y=y10, mode='lines+markers', name=key[9])

In [128]:
decay = [line1, line3]
increase = [line5, line6, line7, line8, line9, line10]
total = [line1, line3, line5, line6, line7, line8, line9, line10]


layout_decay = dict(title = 'topic trend of BMW - decay',
              xaxis = dict(title = 'years'),
              yaxis = dict(title = 'corelation'),
              )
fig_decay = dict(data=decay, layout=layout_decay)


layout_increase = dict(title = 'topic trend of BMW - increase',
              xaxis = dict(title = 'years'),
              yaxis = dict(title = 'corelation'),
              )
fig_increase = dict(data=increase, layout=layout_increase)


layout_total = dict(title = 'all topics trend of BMW',
              xaxis = dict(title = 'years'),
              yaxis = dict(title = 'corelation'),
              )
fig_total = dict(data=total, layout=layout_total)


py.offline.iplot(fig_decay, filename='topic trend of BMW - decay')
py.offline.iplot(fig_increase, filename='topic trend of BMW - increase')
py.offline.iplot(fig_total, filename='all topics trend of BMW')

## Deutsche Bank

In [149]:
key_co = ['Goodwill', 'ABN AMRO', 'Deutsche Bank National Trust Co.', 
          'klage (lawsuit)', 'gericht (judgement)', 'court']
key_ser = ['PCAM', 
           'special purpose entities', 
           'CIB',
           'Incremental Risk Charge',
           'non-core operations unit',
           'exposure at defaut',
           'PWCC',
           'global market'
          ]

In [150]:
y1 = df_db.loc['goodwill'].tolist()
y2 = df_db.loc['amro'].tolist()
y3 = df_db.loc['dbntc'].tolist()
y4 = df_db.loc['klage'].tolist()
y5 = df_db.loc['gericht'].tolist()
y6 = df_db.loc['court'].tolist()

y7 = df_db.loc['pcam'].tolist()
y8 = df_db.loc['spe'].tolist()
y9 = df_db.loc['cib'].tolist()
y10 = df_db.loc['irc'].tolist()
y11 = df_db.loc['ncou'].tolist()
y12 = df_db.loc['ead'].tolist()
y13 = df_db.loc['pwcc'].tolist()
y14 = df_db.loc['gm'].tolist()

In [151]:
years = np.linspace(2010, 2016, 7)

line1 = go.Scatter(x=years, y=y1, mode='lines+markers', name=key_co[0])
line2 = go.Scatter(x=years, y=y2, mode='lines+markers', name=key_co[1])
line3 = go.Scatter(x=years, y=y3, mode='lines+markers', name=key_co[2])
line4 = go.Scatter(x=years, y=y4, mode='lines+markers', name=key_co[3])
line5 = go.Scatter(x=years, y=y5, mode='lines+markers', name=key_co[4])
line6 = go.Scatter(x=years, y=y6, mode='lines+markers', name=key_co[5])

line7 = go.Scatter(x=years, y=y7, mode='lines+markers', name=key_ser[0])
line8 = go.Scatter(x=years, y=y8, mode='lines+markers', name=key_ser[1])
line9 = go.Scatter(x=years, y=y9, mode='lines+markers', name=key_ser[2])
line10 = go.Scatter(x=years, y=y10, mode='lines+markers', name=key_ser[3])
line11 = go.Scatter(x=years, y=y11, mode='lines+markers', name=key_ser[4])
line12 = go.Scatter(x=years, y=y12, mode='lines+markers', name=key_ser[5])
line13 = go.Scatter(x=years, y=y13, mode='lines+markers', name=key_ser[6])
line14 = go.Scatter(x=years, y=y14, mode='lines+markers', name=key_ser[7])

In [152]:
company = [line1, line2, line3, line4, line5, line6]
service = [line7, line8, line9, line10, line11, line12, line13, line14]


layout_company = dict(title = 'Deutsche Bank - topic trend about company',
              xaxis = dict(title = 'years'),
              yaxis = dict(title = 'corelation'),
              )
fig_company = dict(data=company, layout=layout_company)


layout_service = dict(title = 'Deutsche Bank - topic trend about service',
              xaxis = dict(title = 'years'),
              yaxis = dict(title = 'corelation'),
              )
fig_service = dict(data=service, layout=layout_service)

In [153]:
py.offline.iplot(fig_company, filename='Deutsche Bank - topic trend about company')
py.offline.iplot(fig_service, filename='Deutsche Bank - topic trend about service')